In [1]:
from __future__ import print_function, division, absolute_import

import sys

import qminospy
from qminospy.me2 import ME_NLP

# python imports
from copy import copy
import re
from tqdm import tqdm
from os.path import join
from collections import defaultdict
import pickle

# third party imports
import pandas
import cobra

# COBRAme
import cobrame
from cobrame.util import building, mu, me_model_interface
from cobrame.io.json import save_json_me_model, save_reduced_json_me_model

# ECOLIme
import bacillusme
from bacillusme import (transcription, translation, flat_files, generics, formulas, compartments)
from bacillusme.util.helper_functions import *
%load_ext autoreload
%autoreload 2
print(cobra.__file__)
print(cobrame.__file__)
print(bacillusme.__file__)

/home/jt/Downloads/cobrapy-0.5.11/cobra/io/sbml3.py:24: UserWarning: Install lxml for faster SBML I/O
  warn("Install lxml for faster SBML I/O")
/home/jt/Downloads/cobrapy-0.5.11/cobra/io/__init__.py:12: UserWarning: cobra.io.sbml requires libsbml
  warn("cobra.io.sbml requires libsbml")


/home/jt/Downloads/cobrapy-0.5.11/cobra/__init__.py
/home/jt/UCSD/cobrame-master/cobrame/__init__.py
/home/jt/UCSD/bacillusme-master/bacillusme/__init__.py


In [2]:
eco_directory = join(flat_files.ecoli_files_dir, 'iJO1366.json')
ijo_directory = join(flat_files.ecoli_files_dir, 'iYO844.json')
uni_directory = join(flat_files.ecoli_files_dir, 'universal_model.json')

eco = cobra.io.load_json_model(eco_directory)
bsub = cobra.io.load_json_model(ijo_directory)
uni = cobra.io.load_json_model(uni_directory)

eco.optimize()
bsub.optimize()

<Solution 0.12 at 0x7f409eeb32b0>

In [3]:
with open('./me_models/iYO844b.pickle', 'rb') as f:
    me = pickle.load(f)
with open('./me_models/final/iJL1678b.pickle', 'rb') as f:
    ecome = pickle.load(f)

In [ ]:
ecome.metabolites.get_by_id('translation_b4481').reactions

In [ ]:
for r in ecome.reactions:
    if 'SPONT' in r.id:
        print(r.id, r.reaction)

In [ ]:
with open('./me_models/ecoli_solution.pickle', 'rb') as f:
    ecome = pickle.load(f)

In [ ]:
pd.DataFrame.from_dict({'price':ecome.solution.y_dict}).hist(bins=100)

In [ ]:
solve_me_model(me, max_mu = 0.5, min_mu = .05, using_soplex=False, precision = 1e-6)

In [ ]:
solve_me_model(ecome, max_mu = 1., min_mu = .05, using_soplex=False, precision = 1e-6)

In [ ]:
me.solution.y_dict

In [ ]:
me.solution.y_dict

In [ ]:
df = exchange_single_model(ecome)
df.to_csv('temp_result.csv')

In [ ]:
with open('./me_models/solution.pickle', 'wb') as solution:
    pickle.dump(me,solution)

In [ ]:
with open('./me_models/ecoli_solution.pickle', 'wb') as solution:
    pickle.dump(ecome,solution)

# Postprocessing

In [ ]:
with open('./me_models/solution.pickle', 'rb') as solution:
    me = pickle.load(solution)
with open('./me_models/ecoli_solution.pickle', 'rb') as solution:
    ecome = pickle.load(solution)

In [ ]:
b

In [ ]:
me.solution

In [ ]:
# Exchange
df_m = exchange_single_model(bsub)
df = exchange_single_model(me)
df.join(df_m['flux'],lsuffix='_me',rsuffix='_m')

In [ ]:
# Solution summary
summary_df = solution_summary(me)
summary_m_df = solution_summary(bsub)
summary_df.to_csv('./solution_summary.csv')

In [ ]:
summary_df = solution_summary(me)

## Compare M/ME with ecoli

In [ ]:
_,rxn_id_dict = homogenize_reactions(model=bsub,ref_model=eco)

In [ ]:
# M - ME comparison of metabolic fluxes in Bacillus
flux_dict = me.get_metabolic_flux()
me_flux_df = pd.DataFrame.from_dict({'flux':flux_dict}).rename(index=rxn_id_dict)
comparison_df = summary_m_df.join(me_flux_df,lsuffix='_m',rsuffix='_me')
comparison_df[abs(comparison_df.flux_m)>0]

In [ ]:
# M - ME comparison of metabolic fluxes in E. coli
summary_df_ecoli = solution_summary(ecome)
summary_m_df_ecoli = solution_summary(eco)

flux_dict_ecoli = ecome.get_metabolic_flux()
comparison_df_ecoli = summary_m_df_ecoli.join(\
                    pd.DataFrame.from_dict({'flux':flux_dict_ecoli}),lsuffix='_m',rsuffix='_me')

In [ ]:
summary_m_df[summary_m_df.flux > 0]

In [ ]:
import matplotlib.pyplot as plt
threshold = 100
temp_df = comparison_df[abs(comparison_df.flux_m)<threshold]
temp_df_ecoli = comparison_df_ecoli[abs(comparison_df_ecoli.flux_m)<threshold]

plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.scatter(temp_df['flux_m'],temp_df['flux_me'])
plt.xlabel('M simulation')
plt.ylabel('ME simulation')
plt.title('Bacillus')

plt.subplot(1,2,2)
plt.scatter(temp_df_ecoli['flux_m'],temp_df_ecoli['flux_me'])
plt.xlabel('M simulation')
plt.ylabel('ME simulation')
plt.title('E. coli')


In [ ]:
# Store results to visualize in Escher
comparison_df['flux_m'].to_csv('fluxdist_m_bsub.csv',header=False)
comparison_df['flux_me'].to_csv('fluxdist_me_bsub.csv',header=False)

comparison_df_ecoli['flux_m'].to_csv('fluxdist_m_ecoli.csv',header=False)
comparison_df_ecoli['flux_me'].to_csv('fluxdist_me_ecoli.csv',header=False)